In [1]:
def random_symmetric(n):
    import random
    B=[]
    for i in range(1, n+1):
        A=[]
        for j in range(1,n+1):
            if j<i:
                A.append(B[j-1][i-1])
            else:
                A.append(random.randint(1,10**18)/10**18)
        B.append(A)
        C = np.array(B)
    return C

In [2]:
def random_symmetric(n):
    a=[]
    rbf= gaussian_process.kernels.RBF(length_scale=l)
    for i in range(n):
        a.append([random.randint(1,101)/100])
    return rbf(a,a)

In [3]:
from random import Random
from numpy import *
from time import *
from datetime import *
from IPython.display import display
import math
import scipy as sp

import matplotlib.pyplot as plt
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel, RBF

%matplotlib inline
%load_ext autoreload
%autoreload 2

import scipy.stats as st
import numpy as np
import math

def bsformula(cp, s, k, rf, t, v, div):
        """ Price an option using the Black-Scholes model.
        cp: +1/-1 for call/put
        s: initial stock price
        k: strike price
        t: expiration time
        v: volatility
        rf: risk-free rate
        div: dividend
        """

        d1 = (np.log(s/k)+(rf-div+0.5*v*v)*t)/(v*np.sqrt(t))
        d2 = d1 - v*np.sqrt(t)

        optprice = (cp*s*np.exp(-div*t)*st.norm.cdf(cp*d1)) - (cp*k*np.exp(-rf*t)*st.norm.cdf(cp*d2))
        delta = cp*st.norm.cdf(cp*d1)
        vega  = s*np.sqrt(t)*st.norm.pdf(d1)
        return optprice, delta, vega
    
KC = 130
KP = 70
r = 0.002
sigma = 0.4
T = 2.0
S0 = 100

lb = 0
ub = 300
training_number = 3
testing_number = 100

call = lambda x,y: bsformula(1, lb+(ub-lb)*x, KC, r, T, y, 0)[0]
put = lambda x,y: bsformula(-1, lb+(ub-lb)*x, KP, r, T, y, 0)[0]

x_train = np.array(np.linspace(0.01,1.0, training_number), dtype='float32').reshape(training_number, 1)
x_test = np.array(np.linspace(0.01,1.0, testing_number), dtype='float32').reshape(testing_number, 1)

y_train = []
    
for idx in range(len(x_train)):
    y_train.append(call(x_train[idx], sigma))
y_train = np.array(y_train)

sk_kernel = RBF(length_scale=1.0, length_scale_bounds=(0.01, 10000.0)) #+ WhiteKernel(noise_level = 1e-12) #100000.0
gp = gaussian_process.GaussianProcessRegressor(kernel=sk_kernel, n_restarts_optimizer=20)
gp.fit(x_train,y_train)
y_pred, sigma_hat = gp.predict(x_test, return_std=True)
    
l = gp.kernel_.length_scale
rbf= gaussian_process.kernels.RBF(length_scale=l)
 
Kernel= random_symmetric(training_number)
K_y = Kernel + np.eye(training_number) * 1e-8
L, lower = sp.linalg.cho_factor(K_y, lower=True)
L1 = sp.linalg.cholesky(K_y, lower=True)


In [4]:
A1 = L@L.T
display(A1)
display(np.linalg.norm(A1))
B = A1 - K_y
display(np.linalg.norm(B)/np.linalg.norm(A1)) 
#relative error of "Cho_factor" algorithm
#This algorithm is problematic??????? For 3x3 matrix there should be very small error because of the simplicity???

array([[ 2.32534256,  1.91397305,  1.25255105],
       [ 1.91397305,  1.66531187,  0.9249114 ],
       [ 1.25255105,  0.9249114 ,  1.00000001]])

4.6212290382625687

0.51360193395948328

In [5]:
A2 = L1@L1.T
display(A2)
display(np.linalg.norm(A2))
B = A2 - K_y
display(np.linalg.norm(B)/np.linalg.norm(A2)) 
#relative error of "Cholesky" algorithm

array([[ 1.00000001,  0.64243322,  0.95531257],
       [ 0.64243322,  1.00000001,  0.81566651],
       [ 0.95531257,  0.81566651,  1.00000001]])

2.6422166616610059

0.0

In [6]:
K = random_symmetric(3) #Lets use some simple number in a 3x3 matrix for testing
display(K)

array([[ 1.        ,  0.99493325,  0.83287912],
       [ 0.99493325,  1.        ,  0.88074177],
       [ 0.83287912,  0.88074177,  1.        ]])

In [7]:
L = sp.linalg.cholesky(K, lower=True)
display(L)
display(L@L.T)

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.99493325,  0.10053772,  0.        ],
       [ 0.83287912,  0.5180408 ,  0.19479758]])

array([[ 1.        ,  0.99493325,  0.83287912],
       [ 0.99493325,  1.        ,  0.88074177],
       [ 0.83287912,  0.88074177,  1.        ]])

In [8]:
L, lower = sp.linalg.cho_factor(K, lower=True)
display(L)
display(L@L.T)

array([[ 1.        ,  0.99493325,  0.83287912],
       [ 0.99493325,  0.10053772,  0.88074177],
       [ 0.83287912,  0.5180408 ,  0.19479758]])

array([[ 2.6835798 ,  1.828513  ,  1.51053798],
       [ 1.828513  ,  1.77570607,  1.05230813],
       [ 1.51053798,  1.05230813,  1.        ]])